# Value Based Drafting (VBD) Analysis

This notebook calculates VBD using multiple industry-standard methods:
- **VOLS** (Value Over Like Starters): teams × starters
- **VORP** (Value Over Replacement Player): teams × (starters + 1)
- **BEER** (Best Eleven Every Round): teams × (starters + 0.5)
- **Blended**: Weighted combination of all methods

**Input**: Processed projections from `02_basic_rankings.ipynb`
**Output**: Top 300 players ranked by VBD methods

In [10]:
import sys
import os
import pandas as pd
import numpy as np

# Add src directory to path
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

from vbd import calculate_all_vbd_methods, get_top_players_by_vbd, analyze_vbd_distribution
from scoring import load_league_config
from utils import setup_logging, save_rankings, compare_rankings
import logging

# Setup logging
setup_logging()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Load Processed Data

In [11]:
# Load the most recent processed rankings
try:
    # Look for most recent rankings file
    rankings_files = [f for f in os.listdir('../../data/output') if f.startswith('rankings_top300_')]
    if not rankings_files:
        raise FileNotFoundError("No rankings files found. Run 02_basic_rankings.ipynb first.")
    
    latest_file = sorted(rankings_files)[-1]
    df = pd.read_csv(f'../../data/output/{latest_file}')
    
    print(f"Loaded {len(df)} players from {latest_file}")
    print(f"Columns: {list(df.columns)}")
    
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please run 02_basic_rankings.ipynb first to generate processed projections.")

Loaded 300 players from rankings_top300_20250814.csv
Columns: ['PASSING_ATT', 'PASSING_CMP', 'PASSING_YDS', 'PASSING_TDS', 'PASSING_INTS', 'RUSHING_ATT', 'RUSHING_YDS', 'RUSHING_TDS', 'MISC_FL', 'MISC_FPTS', 'POSITION', 'SCRAPE_DATE', 'RECEIVING_REC', 'RECEIVING_YDS', 'RECEIVING_TDS', 'FG', 'FGA', 'XPT', 'FPTS', 'SACK', 'INT', 'FR', 'FF', 'TD', 'SAFETY', 'PA', 'YDS_AGN', 'PLAYER', 'FANTASY_PTS', 'OVERALL_RANK']


## Load League Configuration

In [12]:
# Load league settings
config = load_league_config('../../config/league-config.yaml')

print("League Configuration:")
print(f"Teams: {config.get('basic_settings', {}).get('teams', 'Not specified')}")
print(f"Roster slots: {config.get('roster', {}).get('roster_slots', {})}")

# Show baseline calculations
teams = config.get('basic_settings', {}).get('teams', 12)
roster_slots = config.get('roster', {}).get('roster_slots', {})

print("\nBaseline Calculations (players at replacement level):")
for pos, starters in roster_slots.items():
    vols = teams * starters
    vorp = teams * (starters + 1)
    beer = int(teams * (starters + 0.5))
    print(f"{pos}: VOLS={vols}, VORP={vorp}, BEER={beer}")

League Configuration:
Teams: 14
Roster slots: {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'DEF': 1, 'K': 1}

Baseline Calculations (players at replacement level):
QB: VOLS=14, VORP=28, BEER=21
RB: VOLS=28, VORP=42, BEER=35
WR: VOLS=28, VORP=42, BEER=35
TE: VOLS=14, VORP=28, BEER=21
FLEX: VOLS=14, VORP=28, BEER=21
DEF: VOLS=14, VORP=28, BEER=21
K: VOLS=14, VORP=28, BEER=21


## Calculate VBD Methods

In [13]:
# Calculate all VBD methods
df_vbd = calculate_all_vbd_methods(df, config)

print(f"VBD calculations complete for {len(df_vbd)} players")
print("\nVBD columns added:")
vbd_cols = [col for col in df_vbd.columns if col.startswith('VBD_')]
print(vbd_cols)

2025-08-14 16:39:19,419 - INFO - QB baselines - VOLS: 14, VORP: 28, BEER: 21
2025-08-14 16:39:19,421 - INFO - RB baselines - VOLS: 28, VORP: 42, BEER: 35
2025-08-14 16:39:19,422 - INFO - WR baselines - VOLS: 28, VORP: 42, BEER: 35
2025-08-14 16:39:19,423 - INFO - TE baselines - VOLS: 14, VORP: 28, BEER: 21
2025-08-14 16:39:19,424 - INFO - DST baselines - VOLS: 14, VORP: 28, BEER: 21
2025-08-14 16:39:19,425 - INFO - K baselines - VOLS: 10, VORP: 10, BEER: 10


VBD calculations complete for 300 players

VBD columns added:
['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED']


## Top Players by Each VBD Method

In [14]:
# Show top 20 players by each VBD method
methods = ['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED']

for method in methods:
    print(f"\n=== TOP 20 PLAYERS BY {method} ===")
    top_20 = df_vbd.nlargest(20, method)[['PLAYER', 'POSITION', 'FANTASY_PTS', method]].reset_index(drop=True)
    top_20.index += 1  # Start ranking from 1
    display(top_20)


=== TOP 20 PLAYERS BY VBD_VOLS ===


,PLAYER,POSITION,FANTASY_PTS,VBD_VOLS
1,Saquon Barkley PHI,RB,273.17,128.64
2,Bijan Robinson ATL,RB,262.99,118.46
3,Jahmyr Gibbs DET,RB,259.23,114.70
4,Derrick Henry BAL,RB,248.56,104.03
5,Ja'Marr Chase CIN,WR,231.04,93.65
6,Lamar Jackson BAL,QB,367.52,88.17
7,Josh Allen BUF,QB,366.17,86.82
8,Christian McCaffrey SF,RB,228.67,84.14
9,Josh Jacobs GB,RB,226.58,82.05
10,De'Von Achane MIA,RB,223.46,78.93



=== TOP 20 PLAYERS BY VBD_VORP ===


,PLAYER,POSITION,FANTASY_PTS,VBD_VORP
1,Saquon Barkley PHI,RB,273.17,171.83
2,Bijan Robinson ATL,RB,262.99,161.65
3,Jahmyr Gibbs DET,RB,259.23,157.89
4,Derrick Henry BAL,RB,248.56,147.22
5,Lamar Jackson BAL,QB,367.52,140.53
6,Josh Allen BUF,QB,366.17,139.18
7,Christian McCaffrey SF,RB,228.67,127.33
8,Josh Jacobs GB,RB,226.58,125.24
9,Jalen Hurts PHI,QB,349.57,122.58
10,De'Von Achane MIA,RB,223.46,122.12



=== TOP 20 PLAYERS BY VBD_BEER ===


,PLAYER,POSITION,FANTASY_PTS,VBD_BEER
1,Saquon Barkley PHI,RB,273.17,159.37
2,Bijan Robinson ATL,RB,262.99,149.19
3,Jahmyr Gibbs DET,RB,259.23,145.43
4,Derrick Henry BAL,RB,248.56,134.76
5,Christian McCaffrey SF,RB,228.67,114.87
6,Josh Jacobs GB,RB,226.58,112.78
7,Lamar Jackson BAL,QB,367.52,111.92
8,Josh Allen BUF,QB,366.17,110.57
9,De'Von Achane MIA,RB,223.46,109.66
10,Ja'Marr Chase CIN,WR,231.04,108.70



=== TOP 20 PLAYERS BY VBD_BLENDED ===


,PLAYER,POSITION,FANTASY_PTS,VBD_BLENDED
1,Saquon Barkley PHI,RB,273.17,154.80
2,Bijan Robinson ATL,RB,262.99,144.62
3,Jahmyr Gibbs DET,RB,259.23,140.86
4,Derrick Henry BAL,RB,248.56,130.19
5,Lamar Jackson BAL,QB,367.52,113.13
6,Josh Allen BUF,QB,366.17,111.79
7,Christian McCaffrey SF,RB,228.67,110.30
8,Josh Jacobs GB,RB,226.58,108.21
9,Ja'Marr Chase CIN,WR,231.04,106.10
10,De'Von Achane MIA,RB,223.46,105.09


## VBD Method Comparison

In [15]:
# Compare how different methods rank the same players
comparison = compare_rankings(df_vbd, methods=['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED'], top_n=30)

print("Comparison of Top 30 Rankings Across VBD Methods:")
pivot_comparison = comparison.pivot_table(
    index=['PLAYER', 'POSITION'], 
    columns='Method', 
    values='Rank', 
    fill_value='>30'
)

display(pivot_comparison.head(20))

Comparison of Top 30 Rankings Across VBD Methods:


,Method,VBD_BEER,VBD_BLENDED,VBD_VOLS,VBD_VORP
PLAYER,POSITION,,,,
Alvin Kamara NO,RB,28.0,30.0,>30,29.0
Amon-Ra St. Brown DET,WR,>30,>30,29.0,>30
Ashton Jeanty LV,RB,11.0,11.0,11.0,12.0
Baker Mayfield TB,QB,>30,>30,>30,26.0
Bijan Robinson ATL,RB,2.0,2.0,2.0,2.0
Breece Hall NYJ,RB,25.0,25.0,>30,23.0
Brian Thomas Jr. JAC,WR,>30,>30,28.0,>30
Brock Bowers LV,TE,21.0,21.0,17.0,27.0
Bucky Irving TB,RB,16.0,16.0,19.0,17.0


## Position Distribution Analysis

In [16]:
# Analyze position distribution in top 300 by blended VBD
distribution = analyze_vbd_distribution(df_vbd, top_n=300)

print("Position Distribution in Top 300 (Blended VBD):")
display(distribution)

# Show distribution by different VBD methods
print("\nPosition Distribution Comparison Across Methods:")
for method in ['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED']:
    top_300 = get_top_players_by_vbd(df_vbd, method=method, top_n=300)
    dist = top_300['POSITION'].value_counts()
    print(f"\n{method}:")
    for pos, count in dist.items():
        print(f"  {pos}: {count} ({count/300*100:.1f}%)")

Position Distribution in Top 300 (Blended VBD):


,Count,Percentage
POSITION,,
WR,109,36.3
RB,72,24.0
TE,40,13.3
QB,37,12.3
DST,32,10.7
K,10,3.3



Position Distribution Comparison Across Methods:

VBD_VOLS:
  WR: 109 (36.3%)
  RB: 72 (24.0%)
  TE: 40 (13.3%)
  QB: 37 (12.3%)
  DST: 32 (10.7%)
  K: 10 (3.3%)

VBD_VORP:
  WR: 109 (36.3%)
  RB: 72 (24.0%)
  TE: 40 (13.3%)
  QB: 37 (12.3%)
  DST: 32 (10.7%)
  K: 10 (3.3%)

VBD_BEER:
  WR: 109 (36.3%)
  RB: 72 (24.0%)
  TE: 40 (13.3%)
  QB: 37 (12.3%)
  DST: 32 (10.7%)
  K: 10 (3.3%)

VBD_BLENDED:
  WR: 109 (36.3%)
  RB: 72 (24.0%)
  TE: 40 (13.3%)
  QB: 37 (12.3%)
  DST: 32 (10.7%)
  K: 10 (3.3%)


## Filter to Top 300 and Save

In [17]:
# Get top 300 players by blended VBD
top_300_vbd = get_top_players_by_vbd(df_vbd, method='VBD_BLENDED', top_n=300)

# Add overall VBD rank
top_300_vbd['VBD_RANK'] = range(1, len(top_300_vbd) + 1)

print(f"Top 300 players selected by Blended VBD")
print(f"VBD range: {top_300_vbd['VBD_BLENDED'].max():.2f} to {top_300_vbd['VBD_BLENDED'].min():.2f}")

# Show final rankings
display_cols = ['VBD_RANK', 'PLAYER', 'POSITION', 'FANTASY_PTS', 'VBD_BLENDED', 'VBD_VOLS', 'VBD_VORP', 'VBD_BEER']
print("\nTop 20 by Blended VBD:")
display(top_300_vbd[display_cols].head(20))

Top 300 players selected by Blended VBD
VBD range: 154.80 to -209.70

Top 20 by Blended VBD:


,VBD_RANK,PLAYER,POSITION,FANTASY_PTS,VBD_BLENDED,VBD_VOLS,VBD_VORP,VBD_BEER
0,1,Saquon Barkley PHI,RB,273.17,154.80,128.64,171.83,159.37
1,2,Bijan Robinson ATL,RB,262.99,144.62,118.46,161.65,149.19
2,3,Jahmyr Gibbs DET,RB,259.23,140.86,114.70,157.89,145.43
3,4,Derrick Henry BAL,RB,248.56,130.19,104.03,147.22,134.76
4,5,Lamar Jackson BAL,QB,367.52,113.13,88.17,140.53,111.92
5,6,Josh Allen BUF,QB,366.17,111.79,86.82,139.18,110.57
6,7,Christian McCaffrey SF,RB,228.67,110.30,84.14,127.33,114.87
7,8,Josh Jacobs GB,RB,226.58,108.21,82.05,125.24,112.78
8,9,Ja'Marr Chase CIN,WR,231.04,106.10,93.65,113.37,108.70
9,10,De'Von Achane MIA,RB,223.46,105.09,78.93,122.12,109.66


## Save VBD Rankings

In [18]:
# Save VBD rankings to output
filename = save_rankings(top_300_vbd, filename_prefix="vbd_rankings_top300", output_path="../../data/output")
print(f"VBD rankings saved to: {filename}")

# Also save separate files for each method
for method in ['VBD_VOLS', 'VBD_VORP', 'VBD_BEER']:
    method_top300 = get_top_players_by_vbd(df_vbd, method=method, top_n=300)
    method_filename = save_rankings(
        method_top300, 
        filename_prefix=f"rankings_{method.lower()}_top300", 
        output_path="../../data/output"
    )
    print(f"{method} rankings saved to: {method_filename}")

print("\n✅ VBD analysis complete!")
print("Next steps:")
print("- Use blended VBD rankings for draft preparation")
print("- Compare different methods for positional strategy")
print("- Run 03_draft_strategy.ipynb for draft board analysis")

2025-08-14 16:39:19,493 - INFO - Saved 300 players to vbd_rankings_top300_20250814.csv
2025-08-14 16:39:19,499 - INFO - Saved 300 players to rankings_vbd_vols_top300_20250814.csv
2025-08-14 16:39:19,504 - INFO - Saved 300 players to rankings_vbd_vorp_top300_20250814.csv
2025-08-14 16:39:19,510 - INFO - Saved 300 players to rankings_vbd_beer_top300_20250814.csv


VBD rankings saved to: vbd_rankings_top300_20250814.csv
VBD_VOLS rankings saved to: rankings_vbd_vols_top300_20250814.csv
VBD_VORP rankings saved to: rankings_vbd_vorp_top300_20250814.csv
VBD_BEER rankings saved to: rankings_vbd_beer_top300_20250814.csv

✅ VBD analysis complete!
Next steps:
- Use blended VBD rankings for draft preparation
- Compare different methods for positional strategy
- Run 03_draft_strategy.ipynb for draft board analysis
